# Overview of Prompt Engineering Techniques & Best Practices

## Part 1: Prompt Engineering Best Practices

In this section, we provide an overview of the top tips and best practices for prompting LLMs.

---

We first load the necessary libraries:

In [ ]:
import warnings
import os

import openai
import gradio as gr

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv, find_dotenv
import IPython

In [ ]:
_ = load_dotenv(find_dotenv()) # read local .env file
MODEL = 'gpt-3.5-turbo'
warnings.filterwarnings('ignore')
openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))

In [ ]:
# completion function
def get_completion(messages, model=MODEL, temperature=1, max_tokens=300):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

### Be Specific and Clear

Write instructions as clear and specific as possible to get the desired LLM behaviors:

In [ ]:
global_trending_movies = ["The Suicide Squad", "Bend it like Beckman", "Pele", "No Time to Die", "Dune",  "Spider-Man: No Way Home", "The French Dispatch", "Black Widow", "Eternals", "The Matrix Resurrections", "West Side Story", "The Many Saints of Newark"]

system_message = """
Your task is to recommend movies to a customer.

You are responsible to recommend a movie from the top global trending movies from {global_trending_movies}.

You should refrain from asking users for their preferences and avoid asking for personal information.

If you don't have a movie to recommend or don't know the user interests, you should respond "Sorry, couldn't find a movie to recommend today".
"""

user_request = """
Please recommend a movie based on my interests.
"""

message = [
    {
        "role": "system",
        "content": system_message.format(global_trending_movies=global_trending_movies)
    },
    {
        "role": "user",
        "content": user_request
    }
]

response = get_completion(message)
print(response)

The more specific the desired the behavior you want from the model, the more specific the instructions and logic should be. Below is an example where the customer provides information about interests:

In [ ]:
global_trending_movies = ["The Suicide Squad", "Pele", "Bend it like Beckham", "No Time to Die", "Dune",  "Spider-Man: No Way Home", "The French Dispatch", "Black Widow", "Eternals", "The Matrix Resurrections", "West Side Story", "The Many Saints of Newark"]

system_message = """
Your task is to recommends movies to a customer.

You are responsible to recommend a movie from the top global trending movies from {global_trending_movies}.

You should refrain from asking users for their preferences and avoid asking for personal information.

Provide a short descripition of each movie if more than one. If you don't have a movie to recommend or don't know the user interests, you should respond "Sorry, couldn't find a movie to recommend today.".
"""

user_request = """
I love soccer. Please recommend a movie based on my interests.
"""

message = [
    {
        "role": "system",
        "content": system_message.format(global_trending_movies=global_trending_movies)
    },
    {
        "role": "user",
        "content": user_request
    }
]

response = get_completion(message)
print(response)

### Add Delimiters

Adding delimiters help to better structure instructions and the overall prompt components. This is beneficial to get more reliable responses.

In [ ]:
prompt = """
Convert the following code block in the #### <code> #### section to Python:

####
strings2.push("one")
strings2.push("two")
strings2.push("THREE")
strings2.push("4")
####
"""

message = [
    {
        "role": "user",
        "content": prompt
    }
]

IPython.display.Markdown("```python" + get_completion(message) + "\n```")

### Specify Output Format

If the format of prompt responses are important, then this should be explicitly stated in the prompt to get desired results. In the example, we would like to export the results as a JSON object.

In [ ]:
prompt = """
Your task is: given a product description, return the requested information in the section delimited by ### ###. Format the output as a JSON object.

Product Description: Introducing the Nike Air Max 270 React: a comfortable and stylish sneaker that combines two of Nike's best technologies. With a sleek black design and a unique bubble sole, these shoes are perfect for everyday wear.

###
product_name: the name of the product
product_bran: the name of the brand (if any)
###
"""

message = [
    {
        "role": "user",
        "content": prompt
    }
]

print(get_completion(message))

### Think Step by Step

To elicit reasoning in LLMs, you can prompt the model to think step-by-step. Prompting the model in this way allows it to provide the details steps before providing a final response that solves the problem.

In [ ]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response= get_completion(messages)

print(response)

In [ ]:
prompt = """From these numbers in this group identify all prime numbers: 15, 32, 5, 13, 82, 7, 1.

Solve by breaking the problem into steps. First, identify the prime numbers, add them, and indicate whether the result is odd or even."""

messages = [
    {
        "role": "system",
        "content": prompt
    }
]

response= get_completion(messages)

print(response)

### Role Playing

The example below shows how to apply role playing using a chat model like GPT-3.5 Turbo. Notice the use of system message, user message, and assistant message in the example. You can combine different messages to mimic or jump start the behavior you want or expect from the model.

In [ ]:
system_message = """
The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.
"""

user_message_1 = """
Hello, who are you?
"""

ai_message_1 = """
Greeting! I am an AI research assistant. How can I help you today?
"""

prompt = """
Human: Can you tell me the number of stars in our Milky Way Galaxy?
AI:
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message_1
    },
    {
        "role": "assistant",
        "content": ai_message_1

    },
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(messages)
print(response)